In [35]:
import regex as re

In [141]:
from itertools import chain
from collections import Counter

In [61]:
import pandas as pd

In [43]:
with open('quran-simple-clean.txt', encoding='utf-8') as q_c:
    quran_clean = q_c.readlines()
    quran_clean = [i.strip() for i in quran_clean]

In [44]:
with open('quran-simple-min.txt', encoding='utf-8') as q_m:
    quran_min = q_m.readlines()
    quran_min = [i.strip() for i in quran_min]

In [45]:
char_min = set(''.join(quran_min))

In [46]:
len(char_min)

45

In [47]:
char_clean = set(''.join(quran_clean))

In [48]:
len(char_clean)

37

In [49]:
len(char_min) - len(char_clean)

8

In [50]:
char_min - char_clean

{'ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ٰ'}

In [51]:
tanween = list('''
ٌ
ً
ٍ
'''.strip())

In [ ]:
str.isspace()

In [54]:
tanween = [i for i in tanween if i.isspace() == False]

In [55]:
tanween

['ٌ', 'ً', 'ٍ']

In [56]:
char_clean.update(tanween)

In [57]:
len(char_clean)

40

In [58]:
def clean_unwanted(x):
    result = ''.join([i for i in x if i in char_clean])
    return result

In [59]:
quran_min_cleaned = [clean_unwanted(i) for i in quran_min]

In [123]:
df = pd.DataFrame(data={'Q_CLEAN': quran_clean, 'QURAN_TAN': quran_min_cleaned})

In [124]:
def zipe_sent_words(sent_1, sent_2):
    sent_1 = sent_1.split()
    sent_2 = sent_2.split()
    result = list(zip(sent_1, sent_2))
    return result

In [125]:
df['LEN_C'] = df['Q_CLEAN'].apply(lambda x: len(x.split()))
df['LEN_M'] = df['QURAN_TAN'].apply(lambda x: len(x.split()))

In [126]:
df[df['LEN_C'] != df['LEN_M']]

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M


In [127]:
def remove_bism(x):
    if 'بسم الله الرحمن الرحيم' in x:
        result = re.sub('بسم الله الرحمن الرحيم', '', x)
        return result.strip()
    else:
        result = x
        return result.strip()

In [128]:
df['Q_CLEAN'] = df['Q_CLEAN'].apply(lambda x: remove_bism(x))

In [129]:
df['QURAN_TAN'] = df['QURAN_TAN'].apply(lambda x: remove_bism(x))

In [130]:
df['LEN_C'] = df['Q_CLEAN'].apply(lambda x: len(x.split()))
df['LEN_M'] = df['QURAN_TAN'].apply(lambda x: len(x.split()))

In [113]:
df[df['LEN_C'] != df['LEN_M']]

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M


In [131]:
df = df[df['LEN_C'].apply(lambda x: 2 < x <= 25)]

In [132]:
df = df.reset_index(drop=True)

In [136]:
df = df.iloc[7:].reset_index(drop=True)

In [137]:
df['C_vs_TAN'] = df.apply(lambda x: zipe_sent_words(x['Q_CLEAN'], x['QURAN_TAN']), axis=1)

In [139]:
df['WORDS'] = df['Q_CLEAN'].apply(lambda x: x.split())

In [144]:
dict_words_freq_quran = dict(Counter(list(chain(*list(df['WORDS'])))))

In [147]:
dict_rare_words = {k:v for k,v in dict_words_freq_quran.items() if v < 4}

In [148]:
len(dict_rare_words)

10039

In [149]:
len(dict_words_freq_quran)

12179

In [152]:
'المغضوب' in dict_rare_words

False

In [171]:
dict_words_freq_quran['ختم']

1

In [164]:
#df.to_pickle('QURAN_QUIZZIZ_STAGE_01.pkl', protocol=4)

In [174]:
df = df[df['WORDS'].apply(lambda x: len([i for i in x if i in dict_rare_words])>0)]

In [176]:
df = df.reset_index(drop=True)

In [178]:
#df.to_pickle('QURAN_QUIZZIZ_STAGE_02.pkl', protocol=4)

In [181]:
df['RARE_WORDS'] = df['WORDS'].apply(lambda x: [i for i in x if i in dict_rare_words])

In [184]:
df['LEN_RARE_WORDS'] = df['RARE_WORDS'].apply(lambda x: len(x))

In [186]:
df['RARITY_FACTOR'] = df.apply(lambda x: x['LEN_RARE_WORDS']/x['LEN_C'], axis=1)

In [189]:
df = df.sort_values(['RARITY_FACTOR'], ascending=False).reset_index(drop=True)

In [201]:
#df.to_pickle('QURAN_QUIZZIZ_STAGE_03.pkl')

In [217]:
df_top_900_quiz = df[df['RARITY_FACTOR'] > 0.6]

In [221]:
df_top_900_quiz = df_top_900_quiz.explode('RARE_WORDS')

In [224]:
df_top_900_quiz = df_top_900_quiz.reset_index(drop=True)
df_top_900_quiz.to_pickle('TOP_900_RARE_QURAN_VERSES.pkl', protocol=4)

In [225]:
pd.read_pickle('TOP_900_RARE_QURAN_VERSES.pkl')

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M,C_vs_TAN,WORDS,RARE_WORDS,LEN_RARE_WORDS,RARITY_FACTOR
0,ومناة الثالثة الأخرى,ومناة الثالثة الأخرى,3,3,"[(ومناة, ومناة), (الثالثة, الثالثة), (الأخرى, ...","[ومناة, الثالثة, الأخرى]",ومناة,3,1.000000
1,ومناة الثالثة الأخرى,ومناة الثالثة الأخرى,3,3,"[(ومناة, ومناة), (الثالثة, الثالثة), (الأخرى, ...","[ومناة, الثالثة, الأخرى]",الثالثة,3,1.000000
2,ومناة الثالثة الأخرى,ومناة الثالثة الأخرى,3,3,"[(ومناة, ومناة), (الثالثة, الثالثة), (الأخرى, ...","[ومناة, الثالثة, الأخرى]",الأخرى,3,1.000000
3,كأمثال اللؤلؤ المكنون,كأمثال اللؤلؤ المكنون,3,3,"[(كأمثال, كأمثال), (اللؤلؤ, اللؤلؤ), (المكنون,...","[كأمثال, اللؤلؤ, المكنون]",كأمثال,3,1.000000
4,كأمثال اللؤلؤ المكنون,كأمثال اللؤلؤ المكنون,3,3,"[(كأمثال, كأمثال), (اللؤلؤ, اللؤلؤ), (المكنون,...","[كأمثال, اللؤلؤ, المكنون]",اللؤلؤ,3,1.000000
...,...,...,...,...,...,...,...,...,...
976,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,13,13,"[(وأنا, وأنا), (كنا, كنا), (نقعد, نقعد), (منها...","[وأنا, كنا, نقعد, منها, مقاعد, للسمع, فمن, يست...",يستمع,8,0.615385
977,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,13,13,"[(وأنا, وأنا), (كنا, كنا), (نقعد, نقعد), (منها...","[وأنا, كنا, نقعد, منها, مقاعد, للسمع, فمن, يست...",الآن,8,0.615385
978,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,13,13,"[(وأنا, وأنا), (كنا, كنا), (نقعد, نقعد), (منها...","[وأنا, كنا, نقعد, منها, مقاعد, للسمع, فمن, يست...",يجد,8,0.615385
979,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,وأنا كنا نقعد منها مقاعد للسمع فمن يستمع الآن ...,13,13,"[(وأنا, وأنا), (كنا, كنا), (نقعد, نقعد), (منها...","[وأنا, كنا, نقعد, منها, مقاعد, للسمع, فمن, يست...",شهابا,8,0.615385


In [227]:
df = pd.read_pickle('QURAN_QUIZZIZ_STAGE_03.pkl')

In [229]:
dict_rare_words['الأخرى']

2

In [236]:
def calc_rarity_factor(df):
    factor = df['LEN_RARE_WORDS']/df['LEN_C']
    words = df['WORDS']
    addon = [dict_rare_words.get(i) for i in words]
    addon = [i for i in addon if i==1]
    addon = sum(addon)  * 0.1
    factor = factor + addon
    return factor

In [238]:
df['RARITY_FACTOR'] = df.apply(lambda x: calc_rarity_factor(x), axis=1)

In [244]:
df = df.sort_values(['RARITY_FACTOR'], ascending=False)

In [247]:
df = df.reset_index(drop=True)

In [250]:
df[df['RARITY_FACTOR'] == 0.6]

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M,C_vs_TAN,WORDS,RARE_WORDS,LEN_RARE_WORDS,RARITY_FACTOR
1192,مسومة عند ربك للمسرفين,مسومةً عند ربك للمسرفين,4,4,"[(مسومة, مسومةً), (عند, عند), (ربك, ربك), (للم...","[مسومة, عند, ربك, للمسرفين]","[مسومة, للمسرفين]",2,0.6
1193,الذي يؤتي ماله يتزكى,الذي يؤتي ماله يتزكى,4,4,"[(الذي, الذي), (يؤتي, يؤتي), (ماله, ماله), (يت...","[الذي, يؤتي, ماله, يتزكى]","[يؤتي, يتزكى]",2,0.6
1194,يلبسون من سندس وإستبرق متقابلين,يلبسون من سندسٍ وإستبرقٍ متقابلين,5,5,"[(يلبسون, يلبسون), (من, من), (سندس, سندسٍ), (و...","[يلبسون, من, سندس, وإستبرق, متقابلين]","[يلبسون, سندس, وإستبرق]",3,0.6
1195,وآخر من شكله أزواج,وآخر من شكله أزواجٌ,4,4,"[(وآخر, وآخر), (من, من), (شكله, شكله), (أزواج,...","[وآخر, من, شكله, أزواج]","[وآخر, شكله]",2,0.6
1196,وإن لنا للآخرة والأولى,وإن لنا للآخرة والأولى,4,4,"[(وإن, وإن), (لنا, لنا), (للآخرة, للآخرة), (وا...","[وإن, لنا, للآخرة, والأولى]","[للآخرة, والأولى]",2,0.6
...,...,...,...,...,...,...,...,...,...
1271,ولقد رآه نزلة أخرى,ولقد رآه نزلةً أخرى,4,4,"[(ولقد, ولقد), (رآه, رآه), (نزلة, نزلةً), (أخر...","[ولقد, رآه, نزلة, أخرى]","[رآه, نزلة]",2,0.6
1272,وأتيناك بالحق وإنا لصادقون,وأتيناك بالحق وإنا لصادقون,4,4,"[(وأتيناك, وأتيناك), (بالحق, بالحق), (وإنا, وإ...","[وأتيناك, بالحق, وإنا, لصادقون]","[وأتيناك, لصادقون]",2,0.6
1273,ووجوه يومئذ عليها غبرة,ووجوهٌ يومئذٍ عليها غبرةٌ,4,4,"[(ووجوه, ووجوهٌ), (يومئذ, يومئذٍ), (عليها, علي...","[ووجوه, يومئذ, عليها, غبرة]","[ووجوه, غبرة]",2,0.6
1274,ويبقى وجه ربك ذو الجلال والإكرام,ويبقى وجه ربك ذو الجلال والإكرام,6,6,"[(ويبقى, ويبقى), (وجه, وجه), (ربك, ربك), (ذو, ...","[ويبقى, وجه, ربك, ذو, الجلال, والإكرام]","[ويبقى, الجلال, والإكرام]",3,0.6


In [255]:
df = df[df['RARITY_FACTOR'] > 0.6]

In [256]:
df = df.reset_index(drop=True)

In [258]:
df = df.explode('RARE_WORDS').reset_index(drop=True)

In [261]:
#df.to_pickle('RARE_QURAN_WORDS_RARITY_FACTOR_UPDATED.pkl', protocol=4)

In [262]:
df

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M,C_vs_TAN,WORDS,RARE_WORDS,LEN_RARE_WORDS,RARITY_FACTOR
0,وتحسبهم أيقاظا وهم رقود ونقلبهم ذات اليمين وذا...,وتحسبهم أيقاظًا وهم رقودٌ ونقلبهم ذات اليمين و...,22,22,"[(وتحسبهم, وتحسبهم), (أيقاظا, أيقاظًا), (وهم, ...","[وتحسبهم, أيقاظا, وهم, رقود, ونقلبهم, ذات, الي...",وتحسبهم,14,1.936364
1,وتحسبهم أيقاظا وهم رقود ونقلبهم ذات اليمين وذا...,وتحسبهم أيقاظًا وهم رقودٌ ونقلبهم ذات اليمين و...,22,22,"[(وتحسبهم, وتحسبهم), (أيقاظا, أيقاظًا), (وهم, ...","[وتحسبهم, أيقاظا, وهم, رقود, ونقلبهم, ذات, الي...",أيقاظا,14,1.936364
2,وتحسبهم أيقاظا وهم رقود ونقلبهم ذات اليمين وذا...,وتحسبهم أيقاظًا وهم رقودٌ ونقلبهم ذات اليمين و...,22,22,"[(وتحسبهم, وتحسبهم), (أيقاظا, أيقاظًا), (وهم, ...","[وتحسبهم, أيقاظا, وهم, رقود, ونقلبهم, ذات, الي...",رقود,14,1.936364
3,وتحسبهم أيقاظا وهم رقود ونقلبهم ذات اليمين وذا...,وتحسبهم أيقاظًا وهم رقودٌ ونقلبهم ذات اليمين و...,22,22,"[(وتحسبهم, وتحسبهم), (أيقاظا, أيقاظًا), (وهم, ...","[وتحسبهم, أيقاظا, وهم, رقود, ونقلبهم, ذات, الي...",ونقلبهم,14,1.936364
4,وتحسبهم أيقاظا وهم رقود ونقلبهم ذات اليمين وذا...,وتحسبهم أيقاظًا وهم رقودٌ ونقلبهم ذات اليمين و...,22,22,"[(وتحسبهم, وتحسبهم), (أيقاظا, أيقاظًا), (وهم, ...","[وتحسبهم, أيقاظا, وهم, رقود, ونقلبهم, ذات, الي...",وذات,14,1.936364
...,...,...,...,...,...,...,...,...,...
5538,أأنتم تزرعونه أم نحن الزارعون,أأنتم تزرعونه أم نحن الزارعون,5,5,"[(أأنتم, أأنتم), (تزرعونه, تزرعونه), (أم, أم),...","[أأنتم, تزرعونه, أم, نحن, الزارعون]",الزارعون,2,0.600000
5539,ونزلنا من السماء ماء مباركا فأنبتنا به جنات وح...,ونزلنا من السماء ماءً مباركًا فأنبتنا به جناتٍ...,10,10,"[(ونزلنا, ونزلنا), (من, من), (السماء, السماء),...","[ونزلنا, من, السماء, ماء, مباركا, فأنبتنا, به,...",ونزلنا,4,0.600000
5540,ونزلنا من السماء ماء مباركا فأنبتنا به جنات وح...,ونزلنا من السماء ماءً مباركًا فأنبتنا به جناتٍ...,10,10,"[(ونزلنا, ونزلنا), (من, من), (السماء, السماء),...","[ونزلنا, من, السماء, ماء, مباركا, فأنبتنا, به,...",فأنبتنا,4,0.600000
5541,ونزلنا من السماء ماء مباركا فأنبتنا به جنات وح...,ونزلنا من السماء ماءً مباركًا فأنبتنا به جناتٍ...,10,10,"[(ونزلنا, ونزلنا), (من, من), (السماء, السماء),...","[ونزلنا, من, السماء, ماء, مباركا, فأنبتنا, به,...",وحب,4,0.600000


In [267]:
df['C_vs_TAN'].iloc[0][1][0]

'أيقاظا'

In [296]:
def possible_answers(x):
    tup_01 = x['C_vs_TAN']
    correct_answer = x['RARE_WORDS']
    possible = [i for i in tup_01 if (i[1]  != i[0]) and (i[0] == correct_answer)]
    if possible:
        return possible[0][1]
    else:
        return ''

In [298]:
df['POSSIBLE'] = df.apply(lambda x: possible_answers(x), axis=1)

In [301]:
df.sample(30)

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M,C_vs_TAN,WORDS,RARE_WORDS,LEN_RARE_WORDS,RARITY_FACTOR,POSSIBLE
4790,ونادوا يا مالك ليقض علينا ربك قال إنكم ماكثون,ونادوا يا مالك ليقض علينا ربك قال إنكم ماكثون,9,9,"[(ونادوا, ونادوا), (يا, يا), (مالك, مالك), (لي...","[ونادوا, يا, مالك, ليقض, علينا, ربك, قال, إنكم...",ونادوا,4,0.644444,
5141,وما كان استغفار إبراهيم لأبيه إلا عن موعدة وعد...,وما كان استغفار إبراهيم لأبيه إلا عن موعدةٍ وع...,22,22,"[(وما, وما), (كان, كان), (استغفار, استغفار), (...","[وما, كان, استغفار, إبراهيم, لأبيه, إلا, عن, م...",استغفار,5,0.627273,
1916,فأقبلوا إليه يزفون,فأقبلوا إليه يزفون,3,3,"[(فأقبلوا, فأقبلوا), (إليه, إليه), (يزفون, يزف...","[فأقبلوا, إليه, يزفون]",يزفون,2,0.866667,
3167,ولو دخلت عليهم من أقطارها ثم سئلوا الفتنة لآتو...,ولو دخلت عليهم من أقطارها ثم سئلوا الفتنة لآتو...,14,14,"[(ولو, ولو), (دخلت, دخلت), (عليهم, عليهم), (من...","[ولو, دخلت, عليهم, من, أقطارها, ثم, سئلوا, الف...",سئلوا,5,0.757143,
107,زين للناس حب الشهوات من النساء والبنين والقناط...,زين للناس حب الشهوات من النساء والبنين والقناط...,24,24,"[(زين, زين), (للناس, للناس), (حب, حب), (الشهوا...","[زين, للناس, حب, الشهوات, من, النساء, والبنين,...",المقنطرة,12,1.400000,
2235,إن هذا أخي له تسع وتسعون نعجة ولي نعجة واحدة ف...,إن هذا أخي له تسعٌ وتسعون نعجةً ولي نعجةٌ واحد...,15,15,"[(إن, إن), (هذا, هذا), (أخي, أخي), (له, له), (...","[إن, هذا, أخي, له, تسع, وتسعون, نعجة, ولي, نعج...",الخطاب,8,0.833333,
3741,قال لا تؤاخذني بما نسيت ولا ترهقني من أمري عسرا,قال لا تؤاخذني بما نسيت ولا ترهقني من أمري عسرًا,10,10,"[(قال, قال), (لا, لا), (تؤاخذني, تؤاخذني), (بم...","[قال, لا, تؤاخذني, بما, نسيت, ولا, ترهقني, من,...",عسرا,4,0.700000,عسرًا
401,وإن أردتم استبدال زوج مكان زوج وآتيتم إحداهن ق...,وإن أردتم استبدال زوجٍ مكان زوجٍ وآتيتم إحداهن...,17,17,"[(وإن, وإن), (أردتم, أردتم), (استبدال, استبدال...","[وإن, أردتم, استبدال, زوج, مكان, زوج, وآتيتم, ...",استبدال,8,1.170588,
5095,وجعلها كلمة باقية في عقبه لعلهم يرجعون,وجعلها كلمةً باقيةً في عقبه لعلهم يرجعون,7,7,"[(وجعلها, وجعلها), (كلمة, كلمةً), (باقية, باقي...","[وجعلها, كلمة, باقية, في, عقبه, لعلهم, يرجعون]",باقية,3,0.628571,باقيةً
1266,وأورثكم أرضهم وديارهم وأموالهم وأرضا لم تطئوها...,وأورثكم أرضهم وديارهم وأموالهم وأرضًا لم تطئوه...,13,13,"[(وأورثكم, وأورثكم), (أرضهم, أرضهم), (وديارهم,...","[وأورثكم, أرضهم, وديارهم, وأموالهم, وأرضا, لم,...",وأورثكم,6,0.961538,


In [304]:
#df.to_pickle('QURAN_QUIZZIZ_STAGE_04.pkl', protocol=4)

In [307]:
df.sample(30)

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M,C_vs_TAN,WORDS,RARE_WORDS,LEN_RARE_WORDS,RARITY_FACTOR,POSSIBLE
2563,يدعو لمن ضره أقرب من نفعه لبئس المولى ولبئس ال...,يدعو لمن ضره أقرب من نفعه لبئس المولى ولبئس ال...,10,10,"[(يدعو, يدعو), (لمن, لمن), (ضره, ضره), (أقرب, ...","[يدعو, لمن, ضره, أقرب, من, نفعه, لبئس, المولى,...",العشير,5,0.800000,
2327,أو كصيب من السماء فيه ظلمات ورعد وبرق يجعلون أ...,أو كصيبٍ من السماء فيه ظلماتٌ ورعدٌ وبرقٌ يجعل...,19,19,"[(أو, أو), (كصيب, كصيبٍ), (من, من), (السماء, ا...","[أو, كصيب, من, السماء, فيه, ظلمات, ورعد, وبرق,...",ورعد,8,0.821053,ورعدٌ
26,التائبون العابدون الحامدون السائحون الراكعون ا...,التائبون العابدون الحامدون السائحون الراكعون ا...,16,16,"[(التائبون, التائبون), (العابدون, العابدون), (...","[التائبون, العابدون, الحامدون, السائحون, الراك...",التائبون,10,1.625000,
3004,في صحف مكرمة,في صحفٍ مكرمةٍ,3,3,"[(في, في), (صحف, صحفٍ), (مكرمة, مكرمةٍ)]","[في, صحف, مكرمة]",صحف,2,0.766667,صحفٍ
5332,وإذ قال إبراهيم رب اجعل هذا البلد آمنا واجنبني...,وإذ قال إبراهيم رب اجعل هذا البلد آمنًا واجنبن...,13,13,"[(وإذ, وإذ), (قال, قال), (إبراهيم, إبراهيم), (...","[وإذ, قال, إبراهيم, رب, اجعل, هذا, البلد, آمنا...",وبني,4,0.607692,
3011,وانتظروا إنا منتظرون,وانتظروا إنا منتظرون,3,3,"[(وانتظروا, وانتظروا), (إنا, إنا), (منتظرون, م...","[وانتظروا, إنا, منتظرون]",منتظرون,2,0.766667,
3748,وأشرقت الأرض بنور ربها ووضع الكتاب وجيء بالنبي...,وأشرقت الأرض بنور ربها ووضع الكتاب وجيء بالنبي...,15,15,"[(وأشرقت, وأشرقت), (الأرض, الأرض), (بنور, بنور...","[وأشرقت, الأرض, بنور, ربها, ووضع, الكتاب, وجيء...",ووضع,6,0.700000,
188,الصابرين والصادقين والقانتين والمنفقين والمستغ...,الصابرين والصادقين والقانتين والمنفقين والمستغ...,6,6,"[(الصابرين, الصابرين), (والصادقين, والصادقين),...","[الصابرين, والصادقين, والقانتين, والمنفقين, وا...",والمنفقين,5,1.333333,
2169,يبصرونهم يود المجرم لو يفتدي من عذاب يومئذ ببنيه,يبصرونهم يود المجرم لو يفتدي من عذاب يومئذٍ ببنيه,9,9,"[(يبصرونهم, يبصرونهم), (يود, يود), (المجرم, ال...","[يبصرونهم, يود, المجرم, لو, يفتدي, من, عذاب, ي...",ببنيه,4,0.844444,
4295,أولم ير الذين كفروا أن السماوات والأرض كانتا ر...,أولم ير الذين كفروا أن السماوات والأرض كانتا ر...,18,18,"[(أولم, أولم), (ير, ير), (الذين, الذين), (كفرو...","[أولم, ير, الذين, كفروا, أن, السماوات, والأرض,...",رتقا,5,0.677778,رتقًا


In [310]:
def fill_in_the_gap(x):
    word = x['RARE_WORDS']
    orig = x['Q_CLEAN']
    len_word = len(word)
    gap = '_ ' * len_word
    
    question = re.sub(word, gap, orig)
    return question
    

In [312]:
df['Question_Text'] = df.apply(lambda x: fill_in_the_gap(x), axis=1)

In [315]:
#df.to_pickle('QURAN_QUIZZIZ_STAGE_05.pkl', protocol=4)

In [318]:
quiz = df.rename(columns={
    'Question_Text': 'Question Text', 
    'RARE_WORDS' : 'Option 1', 
    'POSSIBLE': 'Option 2'
})

In [321]:
#quiz.to_pickle('QURAN_QUIZZIZ_STAGE_06.pkl', protocol=4)

In [324]:
quiz.head(100).sample(30)

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M,C_vs_TAN,WORDS,Option 1,LEN_RARE_WORDS,RARITY_FACTOR,Option 2,Question Text
13,وتحسبهم أيقاظا وهم رقود ونقلبهم ذات اليمين وذا...,وتحسبهم أيقاظًا وهم رقودٌ ونقلبهم ذات اليمين و...,22,22,"[(وتحسبهم, وتحسبهم), (أيقاظا, أيقاظًا), (وهم, ...","[وتحسبهم, أيقاظا, وهم, رقود, ونقلبهم, ذات, الي...",رعبا,14,1.936364,رعبًا,وتحسبهم أيقاظا وهم رقود ونقلبهم ذات اليمين وذا...
92,وضرب الله مثلا قرية كانت آمنة مطمئنة يأتيها رز...,وضرب الله مثلًا قريةً كانت آمنةً مطمئنةً يأتيه...,24,24,"[(وضرب, وضرب), (الله, الله), (مثلا, مثلًا), (ق...","[وضرب, الله, مثلا, قرية, كانت, آمنة, مطمئنة, ي...",آمنة,12,1.400000,آمنةً,وضرب الله مثلا قرية كانت _ _ _ _ مطمئنة يأتيه...
89,إيلافهم رحلة الشتاء والصيف,إيلافهم رحلة الشتاء والصيف,4,4,"[(إيلافهم, إيلافهم), (رحلة, رحلة), (الشتاء, ال...","[إيلافهم, رحلة, الشتاء, والصيف]",الشتاء,4,1.400000,,إيلافهم رحلة _ _ _ _ _ _ والصيف
37,ملعونين أينما ثقفوا أخذوا وقتلوا تقتيلا,ملعونين أينما ثقفوا أخذوا وقتلوا تقتيلًا,6,6,"[(ملعونين, ملعونين), (أينما, أينما), (ثقفوا, ث...","[ملعونين, أينما, ثقفوا, أخذوا, وقتلوا, تقتيلا]",أينما,6,1.600000,,ملعونين _ _ _ _ _ ثقفوا أخذوا وقتلوا تقتيلا
75,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...,16,16,"[(الخبيثات, الخبيثات), (للخبيثين, للخبيثين), (...","[الخبيثات, للخبيثين, والخبيثون, للخبيثات, والط...",للطيبين,9,1.462500,,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...
41,ملعونين أينما ثقفوا أخذوا وقتلوا تقتيلا,ملعونين أينما ثقفوا أخذوا وقتلوا تقتيلًا,6,6,"[(ملعونين, ملعونين), (أينما, أينما), (ثقفوا, ث...","[ملعونين, أينما, ثقفوا, أخذوا, وقتلوا, تقتيلا]",تقتيلا,6,1.600000,تقتيلًا,ملعونين أينما ثقفوا أخذوا وقتلوا _ _ _ _ _ _
43,أن اقذفيه في التابوت فاقذفيه في اليم فليلقه ال...,أن اقذفيه في التابوت فاقذفيه في اليم فليلقه ال...,22,22,"[(أن, أن), (اقذفيه, اقذفيه), (في, في), (التابو...","[أن, اقذفيه, في, التابوت, فاقذفيه, في, اليم, ف...",التابوت,11,1.600000,,أن اقذفيه في _ _ _ _ _ _ _ فاقذفيه في اليم فل...
76,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...,16,16,"[(الخبيثات, الخبيثات), (للخبيثين, للخبيثين), (...","[الخبيثات, للخبيثين, والخبيثون, للخبيثات, والط...",والطيبون,9,1.462500,,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...
70,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...,الخبيثات للخبيثين والخبيثون للخبيثات والطيبات ...,16,16,"[(الخبيثات, الخبيثات), (للخبيثين, للخبيثين), (...","[الخبيثات, للخبيثين, والخبيثون, للخبيثات, والط...",الخبيثات,9,1.462500,,_ _ _ _ _ _ _ _ للخبيثين والخبيثون للخبيثات و...
14,فأعرضوا فأرسلنا عليهم سيل العرم وبدلناهم بجنتي...,فأعرضوا فأرسلنا عليهم سيل العرم وبدلناهم بجنتي...,16,16,"[(فأعرضوا, فأعرضوا), (فأرسلنا, فأرسلنا), (عليه...","[فأعرضوا, فأرسلنا, عليهم, سيل, العرم, وبدلناهم...",فأعرضوا,12,1.650000,,_ _ _ _ _ _ _ فأرسلنا عليهم سيل العرم وبدلناه...


In [328]:
test = pd.read_pickle('QURAN_QUIZZIZ_STAGE_05.pkl')

In [330]:
test[test['RARE_WORDS'].apply(lambda x: 'حملناه' in x)]

,Q_CLEAN,QURAN_TAN,LEN_C,LEN_M,C_vs_TAN,WORDS,RARE_WORDS,LEN_RARE_WORDS,RARITY_FACTOR,POSSIBLE,Question_Text
1612,وحملناه على ذات ألواح ودسر,وحملناه على ذات ألواحٍ ودسرٍ,5,5,"[(وحملناه, وحملناه), (على, على), (ذات, ذات), (...","[وحملناه, على, ذات, ألواح, ودسر]",وحملناه,3,0.9,,_ _ _ _ _ _ _ على ذات ألواح ودسر


In [ ]:
df